# Implementing and vectorizing a Maximum Likelihood model with scipy

**Matt Ranger**

This notebook walks through the process of coding, estimating, and vectorizing a Maximum Likelihood (MLE) model "from scratch" using scipy's numerical optimization package. Not all MLE models are available in pre-cooked packages, so this skill is necessary for some research topics.

We will only be touching a simple model here (regular probit), so as not to get bogged down in extraneous details. This method of estimation and vectorization will work for a wide range of MLE models.

**Probit**

The basic probit model takes a binary dependent variable, and assumes that $PR(Y=1 | X) = \Phi(X^T \beta)$ where $X$ is the matrix of independent variables, $\beta$ is the vector of parameters, and $\Phi$ is the CDF of the standard normal distribution. We want to take the likelihood function $L=PR(\beta|X, Y)$, and maximize it over $\beta$ to get the most likely $\beta$ paramaters given the data $X,Y$. We usually use the log of the likelihood function in practice, because it is simpler computationally, and the maximum point is the same. The probit log likelihood is as follows:

$$ln L(\beta|X) = \sum_{i=1}^n[y_i ln \Phi(x_i'\beta)+(1-y_i)ln(1-\Phi(x_i'\beta)) ]$$

Which we can translate into a naive python function like this:

In [1]:
import numpy as np
from scipy.stats import norm

def LogLikeProbit(betas, y, x):
    """
    Probit Log Likelihood function
    Very slow naive Python version
    Input:
        betas is a np.array of parameters
        y is a one dimensional np.array of endogenous data
        x is a 2 dimensional np.array of exogenous data
        First vertical colmn of X is assumed to be constant term,
        as is betas[0]
    returns:
        log likehood value (scalar)
    """
    result = 0
    #Sum operation
    for i in range(0, len(y)):
        #Get X_i * Beta value
        xb = np.dot(x[i], betas)
        
        #compute both binary probabilities from xb     
        #Add to total log likelihood
        llf = y[i]*np.log(norm.cdf(xb)) + (1-y[i])*np.log(1 - norm.cdf(xb))
        result += llf
    return -result


Note that we return the negative value of the result because we want to maximize over this function, and numerical optimizers are traditionally minimizers. Minimizing over the negative values will be the same as maximizing the function.

Here is artificial data to test it on:

In [2]:
######################
#ARTIFICIAL DATA
######################

#sample size
n = 1000

#generators
z1 = np.random.randn(n)
z2 = np.random.randn(n)

#create artificial exogenous variables
x1 = 0.8*z1 + 0.2*z2
x2 = 0.2*z1 + 0.8*z2
#error term
u = 2*np.random.randn(n)

#create LHS variable from above
ystar = 0.5 + 0.75*x1 - 0.75*x2 + u

#create latent binary variable from ystar
def create_dummy(data, cutoff):
    result = np.zeros(len(data))
    for i in range(0, len(data)):
        if data[i] >= cutoff:
            result[i] = 1
        else:
            result[i] = 0
    return result

#get latent LHS variable
y = create_dummy(ystar, 0.25)

#prepend vector of ones to RHS variables matrix
const = np.ones(n)
x = np.column_stack((const, np.column_stack((x1, x2))))



We can now maximize the probit log likelihood to get the most likely vector of parameters given the artificial data using scipy's powerful numerical optimization library:

In [3]:
from scipy.optimize import minimize

#create beta hat vector to maximize on
#will store the values of maximum likelihood beta parameters
#Arbitrarily initialized to all zeros
bhat = np.zeros(len(x[0]))

#unvectorized MLE estimation
probit_est = minimize(LogLikeProbit, bhat, args=(y,x), method='nelder-mead')

probit_est['x']

array([ 0.13116994,  0.48000114, -0.47719815])

Note here that probit results can't be interpreted directly like linear regression results. Parameter estimates are divided by $\sigma$ (which we set to 2 when generating the error term). You can multiply the last two values of 'x' in the results displayed by $\sigma$ and compare that to our generated values of x1 and x2 (0.75 and -0.75 respectively).

There are many algorithms in scipy.optimize.minimize's module; [this is a good reference to help choose the best one](http://www.scipy-lectures.org/advanced/mathematical_optimization/).If in doubt, the [Nelder-Mead](http://nbviewer.jupyter.org/github/QuantEcon/QuantEcon.notebooks/blob/master/chase_nelder_mead.ipynb) algorithm is included in scipy's minimizer and is usually a good choice. Nelder-Mead doesn't require estimating derivatives of the function, and as such fails less often, at a cost of being slow to converge on larger datasets.

This is a good time to get some summary statistics on our empirical estimate. In maximum likelihood estimation, the standard error is usually computed from the [Cramèr-Rao lower bound](https://www.youtube.com/watch?v=i0JiSddCXMM). We can then use the standard error to get t- and p-values. The C-R lower bound is computed by the square root of the diagonal elements of the inverse of the Hessian at our estimated parameters. [Statsmodels'](https://github.com/statsmodels/statsmodels) numerical differentiation toolbox makes this easy:

In [10]:
import statsmodels.tools.numdiff as smt
import scipy as sc

#Get inverse hessian for Cramer Rao lower bound
b_estimates = probit_est['x']
Hessian = smt.approx_hess3(b_estimates, LogLikeProbit, args=(y,x))
invHessian = np.linalg.inv(Hessian)

#Standard Errors from C-R LB
#from diagonal elements of invHessian
SE = np.zeros(len(invHessian))
for i in range(0, len(invHessian)):
    SE[i] =  np.sqrt(invHessian[i,i])
    
#t and p values
t_statistics = (b_estimates/SE)
pval = (sc.stats.t.sf(np.abs(t_statistics), 999)*2)

print(b_estimates)
print(SE)
print(t_statistics)
print(pval)

[ 0.02393004  0.41082229 -0.41999823]
[ 0.04054334  0.05668273  0.05728756]
[ 0.59023361  7.24775059 -7.33140354]
[  5.55167525e-01   8.47081591e-13   4.69796690e-13]


Now that we know this probit model works, it would be a good time to vectorize the naive python into a performant version.

The naive model is very inefficient. More complex models will require nontrivial computational power to estimate, and proper vectorization can easily reduce a computation from taking hours to taking minutes.

The main idea is to replace as much computation from "pure python" to optimized numpy/scipy functions. To do this we need to look closely at what our code is doing. Here is the main loop in the naive probit model:

The outer loop is actually just a $\sum_{i=0}^n$ operation, so we can replace the outer for loop by numpy's optimized sum function. To do this, we have to make a couple of changes to the code. 

- First, we replace the for loop by wrapping np.sum() around the code inside the loop. 

- Move the $X_i'\beta$ computation outside the loop to run it only once.

- Relace the conditional $y_i$ and $(1-y_i)$ in the loop by pythonic conditionals that are allowed inside the sum() function. The (y==1) and (y==0) conditional expressions can do this inside the sum function.

For further optimization, we can use the natural log's mathematical propreties and move the log calculation to the outer part of the loop, so it's calculated once for the entire sum, instead of at each observation:

So the new vectorized log likelihood function looks like this:

In [4]:
def VectorizedProbitLL(betas, y, x):
    xb = np.dot(x, betas)
    result = np.sum(np.log(  
        (y==0)*(1 - norm.cdf(xb)) + 
        (y==1)*(norm.cdf(xb))
        ))
    return -result

Again, we return the negative value of the sum because optimization libraries generally minimize, and we're trying to maximize. We'll see a drastic difference in runtime right away:

In [7]:
import timeit

%timeit minimize(VectorizedProbitLL, bhat, args=(y,x), method='nelder-mead')

10 loops, best of 3: 43.6 ms per loop


In [6]:
%timeit minimize(LogLikeProbit, bhat, args=(y,x), method='nelder-mead')

1 loop, best of 3: 17.2 s per loop


So the vectorized version is 400-500x faster (!!!) with the Nelder-Mead algorithm. Vectorized code also tends to scale better to multicore processors. Note that while 17 seconds is not a problem on our trivial sample size (n=1000 in the artificial data above), unvectorized code can become a serious problem on large datasets.


We can also compare the performance of minimization algorithms:

In [13]:
%timeit minimize(VectorizedProbitLL, bhat, args=(y,x), method='bfgs')

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: divide by zero encountered in log


10 loops, best of 3: 25.2 ms per loop


Note that the BFGS algorithm throws a warning about a division by 0 here. This most likely comes from the fact that all our data is binary; methods that have to estimate derivatives have a harder time with sparse or binary data (among others).